In [1]:
import pandas as pd

In [39]:
gun_violence_filepath = "gunviolence.pickle"
policy_filepath = "policy_metadata.csv"

gun_violence_df = pd.read_pickle(gun_violence_filepath)
policy_df = pd.read_csv(policy_filepath)

In [42]:
gun_violence_df[:5]

,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,...,participant_type,sources,state_house_district,state_senate_district,year,suicide,mass shooting,gang,wounded,dead
0,278,95289,2014-01-01,Michigan,Muskegon,300 block of Monroe Avenue,0,0,http://www.gunviolencearchive.org/incident/95289,http://www.mlive.com/news/muskegon/index.ssf/2...,...,0::Victim,http://www.mlive.com/news/muskegon/index.ssf/2...,92.0,34.0,2014,False,False,False,False,False
1,279,92401,2014-01-01,New Jersey,Newark,Central Avenue,0,0,http://www.gunviolencearchive.org/incident/92401,http://www.nj.com/essex/index.ssf/2014/01/newa...,...,NaN,http://www.nj.com/essex/index.ssf/2014/01/newa...,29.0,29.0,2014,False,False,False,False,False
2,280,92383,2014-01-01,New York,Queens,113th Avenue,1,0,http://www.gunviolencearchive.org/incident/92383,http://www.timesledger.com/stories/2014/2/firs...,...,0::Victim||1::Subject-Suspect,http://www.timesledger.com/stories/2014/2/firs...,33.0,14.0,2014,True,False,False,False,True
3,281,92142,2014-01-01,New York,Brooklyn,St. Johns Place,0,1,http://www.gunviolencearchive.org/incident/92142,http://www.nydailynews.com/new-york/nyc-crime/...,...,0::Victim||1::Subject-Suspect,http://www.nydailynews.com/new-york/nyc-crime/...,43.0,20.0,2014,False,False,False,True,False
4,282,95261,2014-01-01,Missouri,Springfield,Beverly Hills and Temple,0,1,http://www.gunviolencearchive.org/incident/95261,http://www.ozarksfirst.com/story/deputies-6-ye...,...,0::Victim||1::Subject-Suspect,http://www.ozarksfirst.com/story/deputies-6-ye...,131.0,30.0,2014,False,False,False,True,False


In [40]:
policy_df[:5]

,year,state,category,sub_category,policies_implemented
0,1991,Alabama,Ammunition regulations,Prohibitors,0
1,1991,Alabama,Ammunition regulations,Background checks,0
2,1991,Alabama,Assault weapons and large-capacity magazines,Assault weapons ban,0
3,1991,Alabama,Assault weapons and large-capacity magazines,Large capacity magazine ban,0
4,1991,Alabama,Background checks,Universal background checks,0


In [51]:
set(policy_df.category)

{'Ammunition regulations',
 'Assault weapons and large-capacity magazines',
 'Background checks',
 'Buyer regulations',
 'Child access prevention',
 'Dealer regulations',
 'Domestic violence',
 'Gun trafficking',
 'Immunity',
 'Possession regulations',
 'Preemption',
 'Prohibitions for high-risk gun possession',
 'Stand your ground'}

# Correlation Between Total # Incidents and Policies

In [58]:
all_incidents = pd.DataFrame(gun_violence_df.groupby(['year', 'state']).size()).join(
                                policy_df[['year', 'state','category', 'policies_implemented']].groupby(
                                    ['year', 'state', 'category']).sum())

all_incidents.rename(columns={0:'gun_violence_incidents'}, inplace=True)
all_incidents.reset_index(inplace=True)
all_incidents

,year,state,category,gun_violence_incidents,policies_implemented
0,2014,Alabama,Ammunition regulations,1318,3
1,2014,Alabama,Assault weapons and large-capacity magazines,1318,0
2,2014,Alabama,Background checks,1318,0
3,2014,Alabama,Buyer regulations,1318,1
4,2014,Alabama,Child access prevention,1318,0
...,...,...,...,...,...
3245,2018,Wyoming,Immunity,14,1
3246,2018,Wyoming,Possession regulations,14,3
3247,2018,Wyoming,Preemption,14,0
3248,2018,Wyoming,Prohibitions for high-risk gun possession,14,1


In [71]:
policy_correlations = {'category': [], 'incidence_type': [], 'corr': []}
for category in set(all_incidents.category):
    df_category = all_incidents[all_incidents.category == category]
    df_category_corr = df_category.corr()
    policy_correlations['category'].append(category)
    policy_correlations['incidence_type'].append('all_incidents')
    policy_correlations['corr'].append(df_category_corr.iloc[1,2])
    
    print(f'correlation with {category} policies is {df_category_corr.iloc[1,2]}')

correlation with Background checks policies is 0.17445044675465265
correlation with Child access prevention policies is 0.275075033451633
correlation with Domestic violence policies is 0.2778645683277723
correlation with Buyer regulations policies is 0.19794442541070792
correlation with Immunity policies is 0.007159513586771926
correlation with Possession regulations policies is 0.5002984759379122
correlation with Dealer regulations policies is 0.22126928802351672
correlation with Ammunition regulations policies is 0.3572870846004122
correlation with Gun trafficking policies is 0.37352904815706656
correlation with Prohibitions for high-risk gun possession policies is 0.11610194530235231
correlation with Preemption policies is 0.13565868789467284
correlation with Assault weapons and large-capacity magazines policies is 0.24437063302029147
correlation with Stand your ground policies is -0.06581047526349888


Possession regulations policies have a correlation of 0.5!

# Correlation Between Total # Suicide Incidents and Total # Policies

In [72]:
suicides = pd.DataFrame(gun_violence_df[gun_violence_df.suicide].groupby(['year', 'state']).size()).join(
                                policy_df[['year', 'state','category', 'policies_implemented']].groupby(
                                    ['year', 'state', 'category']).sum())

suicides.rename(columns={0:'gun_violence_incidents'}, inplace=True)
suicides.reset_index(inplace=True)

for category in set(suicides.category):
    df_category = suicides[suicides.category == category]
    df_category_corr = df_category.corr()
    policy_correlations['category'].append(category)
    policy_correlations['incidence_type'].append('suicides')
    policy_correlations['corr'].append(df_category_corr.iloc[1,2])
    
    print(f'correlation with {category} policies is {df_category_corr.iloc[1,2]}')

correlation with Background checks policies is 0.10421938698819337
correlation with Child access prevention policies is 0.23293547869769018
correlation with Domestic violence policies is 0.19324068582621998
correlation with Buyer regulations policies is 0.1273084069763417
correlation with Immunity policies is -0.08411560948403683
correlation with Possession regulations policies is 0.40298201927992305
correlation with Dealer regulations policies is 0.15513663198467129
correlation with Ammunition regulations policies is 0.2713966916566224
correlation with Gun trafficking policies is 0.317989865868259
correlation with Prohibitions for high-risk gun possession policies is 0.038285607785568186
correlation with Preemption policies is 0.040735307741119185
correlation with Assault weapons and large-capacity magazines policies is 0.20336977264016162
correlation with Stand your ground policies is -0.15452427506075653


# Correlation Between Total # Non-Suicide Incidents and Total # Policies

In [73]:
non_suicides = pd.DataFrame(gun_violence_df[gun_violence_df.suicide != True].groupby(['year', 'state']).size()).join(
                                policy_df[['year', 'state','category', 'policies_implemented']].groupby(
                                    ['year', 'state', 'category']).sum())

non_suicides.rename(columns={0:'gun_violence_incidents'}, inplace=True)
non_suicides.reset_index(inplace=True)

for category in set(non_suicides.category):
    df_category = non_suicides[non_suicides.category == category]
    df_category_corr = df_category.corr()
    policy_correlations['category'].append(category)
    policy_correlations['incidence_type'].append('non_suicides')
    policy_correlations['corr'].append(df_category_corr.iloc[1,2])
    
    print(f'correlation with {category} policies is {df_category_corr.iloc[1,2]}')

correlation with Background checks policies is 0.19374660702579527
correlation with Child access prevention policies is 0.2823863844174031
correlation with Domestic violence policies is 0.29942102098877815
correlation with Buyer regulations policies is 0.21678878455099296
correlation with Immunity policies is 0.03773508766743152
correlation with Possession regulations policies is 0.5205622686014983
correlation with Dealer regulations policies is 0.23801223864179932
correlation with Ammunition regulations policies is 0.3772816745006713
correlation with Gun trafficking policies is 0.38289023074162354
correlation with Prohibitions for high-risk gun possession policies is 0.1394148890602985
correlation with Preemption policies is 0.1642461217640074
correlation with Assault weapons and large-capacity magazines policies is 0.25206709250455883
correlation with Stand your ground policies is -0.03427088178487527


# Correlation Between Total # Mass Shooting Incidents and Total # Policies

In [74]:
mass_shooting = pd.DataFrame(gun_violence_df[gun_violence_df['mass shooting']].groupby(['year', 'state']).size()).join(
                                policy_df[['year', 'state','category', 'policies_implemented']].groupby(
                                    ['year', 'state', 'category']).sum())

mass_shooting.rename(columns={0:'gun_violence_incidents'}, inplace=True)
mass_shooting.reset_index(inplace=True)

for category in set(mass_shooting.category):
    df_category = mass_shooting[mass_shooting.category == category]
    df_category_corr = df_category.corr()
    policy_correlations['category'].append(category)
    policy_correlations['incidence_type'].append('mass_shooting')
    policy_correlations['corr'].append(df_category_corr.iloc[1,2])
    
    print(f'correlation with {category} policies is {df_category_corr.iloc[1,2]}')

correlation with Background checks policies is 0.1848063213309072
correlation with Child access prevention policies is 0.27892827051644187
correlation with Domestic violence policies is 0.3163652924097283
correlation with Buyer regulations policies is 0.2446676378773229
correlation with Immunity policies is 0.11881449514271315
correlation with Possession regulations policies is 0.4677932023752504
correlation with Dealer regulations policies is 0.22250400285099897
correlation with Ammunition regulations policies is 0.23051071906275442
correlation with Gun trafficking policies is 0.39121201614826234
correlation with Prohibitions for high-risk gun possession policies is 0.13467095961991718
correlation with Preemption policies is 0.17804086313925185
correlation with Assault weapons and large-capacity magazines policies is 0.21847350091537907
correlation with Stand your ground policies is 0.016188458406622433


# Creating Correlation DataFrame

In [77]:
policy_correlation_df = pd.DataFrame(policy_correlations).pivot(index='category', columns='incidence_type', values='corr')
policy_correlation_df

incidence_type,all_incidents,mass_shooting,non_suicides,suicides
category,,,,
Ammunition regulations,0.357287,0.230511,0.377282,0.271397
Assault weapons and large-capacity magazines,0.244371,0.218474,0.252067,0.203370
Background checks,0.174450,0.184806,0.193747,0.104219
Buyer regulations,0.197944,0.244668,0.216789,0.127308
Child access prevention,0.275075,0.278928,0.282386,0.232935
Dealer regulations,0.221269,0.222504,0.238012,0.155137
Domestic violence,0.277865,0.316365,0.299421,0.193241
Gun trafficking,0.373529,0.391212,0.382890,0.317990
Immunity,0.007160,0.118814,0.037735,-0.084116


In [78]:
policy_correlation_df.to_csv('policy_incidence_correlations.csv')